In [1]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

### Regras de associação

In [2]:
def gerar_regras_associacao(df, cod_produto, min_support=0.01, min_threshold=3, max_len=2):
    """    
    Parâmetros:
    - df: DataFrame com os dados
    - cod_produto: Código do produto alvo
    - min_support: Suporte mínimo aumentado (padrão 0.05)
    - min_threshold: Limite mínimo para regras
    - max_len: Tamanho máximo dos itemsets (padrão 2 para reduzir complexidade)
    """

    
    # 1. Filtrar apenas notas fiscais que contêm o produto alvo
    notas_com_produto = df[df['Código produto'] == cod_produto]['Numero nota fiscal'].unique()
    df_filtrado = df[df['Numero nota fiscal'].isin(notas_com_produto)]
    
    # 2. Agrupar produtos por nota fiscal
    transacoes = df_filtrado.groupby('Numero nota fiscal')['Código produto'].apply(list)
    
    # 3. Converter para one-hot encoding
    te = TransactionEncoder()
    te_ary = te.fit_transform(transacoes)
    df_onehot = pd.DataFrame(te_ary, columns=te.columns_)
    
    # 4. Aplicar Apriori com parâmetros conservadores
    frequent_itemsets = apriori(df_onehot, 
                              min_support=min_support, 
                              use_colnames=True,
                              max_len=max_len)  # Limita o tamanho dos itemsets
    
    if frequent_itemsets.empty:
        return pd.DataFrame()  # Retorna vazio se não encontrar itemsets frequentes
    
    # 5. Gerar regras de associação
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=min_threshold)
    
    # 6. Filtrar apenas regras que contenham o produto alvo
    produto_str = str(cod_produto)
    rules_produto = rules[
        rules['antecedents'].apply(lambda x: produto_str in str(x)) |
        rules['consequents'].apply(lambda x: produto_str in str(x))
    ]
    
    return rules_produto


# Carregar dados
caminho_arquivo = pd.read_csv("bases/bases_limpas/base_para_modelo_final.csv")

# Código do produto
codigo = 36952

# Chamada mais segura
regras = gerar_regras_associacao(
    caminho_arquivo,
    cod_produto=codigo,
    min_support=0.1,  # Valor mais alto
    max_len=3      # Limita combinações
)

regras.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
2324,"(36952, 1572)",(1573),0.111111,0.111111,0.111111,1.0,9.0,1.0,0.098765,inf,1.0,1.0,1.0,1.0
2325,"(36952, 1573)",(1572),0.111111,0.111111,0.111111,1.0,9.0,1.0,0.098765,inf,1.0,1.0,1.0,1.0
2326,(1572),"(36952, 1573)",0.111111,0.111111,0.111111,1.0,9.0,1.0,0.098765,inf,1.0,1.0,1.0,1.0
2327,(1573),"(36952, 1572)",0.111111,0.111111,0.111111,1.0,9.0,1.0,0.098765,inf,1.0,1.0,1.0,1.0
2380,"(36952, 1572)",(4303),0.111111,0.111111,0.111111,1.0,9.0,1.0,0.098765,inf,1.0,1.0,1.0,1.0


### Formatando as regras para serem exibidas de forma mais amigável

In [3]:
def formatar_regras_com_info(df_regras, df_original):
    """
    Formata o DataFrame de regras de associação com informações completas dos produtos
    
    Parâmetros:
    - df_regras: DataFrame com as regras de associação (saída do algoritmo)
    - df_original: DataFrame original com informações dos produtos
    
    Retorna:
    - DataFrame formatado com informações completas
    """
    # Criar dicionário com informações dos produtos
    produtos_info = df_original.drop_duplicates('Código produto').set_index('Código produto')[
        ['Descrição do produto', 'Valor unitário', 'Margem %', 'Quantidade estoque']
    ].to_dict('index')
    
    # Função para extrair o primeiro código do itemset
    def extrair_codigo(itemset):
        return next(iter(itemset))
    
    # Função para obter informações do produto
    def get_produto_info(codigo):
        info = produtos_info.get(codigo, {
            'Descrição do produto': 'PRODUTO NÃO ENCONTRADO',
            'Valor unitário': 0,
            'Margem %': 0,
            'Quantidade estoque': 0
        })
        return info
    
    # Processar cada regra
    dados_formatados = []
    for _, regra in df_regras.iterrows():
        # Obter códigos dos produtos
        cod_antec = extrair_codigo(regra['antecedents'])
        cod_conseq = extrair_codigo(regra['consequents'])
        
        # Obter informações dos produtos
        info_antec = get_produto_info(cod_antec)
        info_conseq = get_produto_info(cod_conseq)
        
        # Adicionar ao resultado formatado
        dados_formatados.append({
            'Antecedente (Código)': cod_antec,
            'Descrição Antecedente': info_antec['Descrição do produto'],
            'Valor Unitário Antecedente': info_antec['Valor unitário'],
            'Margem % Antecedente': info_antec['Margem %'],
            'Estoque Antecedente': info_antec['Quantidade estoque'],
            
            'Consequente (Código)': cod_conseq,
            'Descrição Consequente': info_conseq['Descrição do produto'],
            'Valor Unitário Consequente': info_conseq['Valor unitário'],
            'Margem % Consequente': info_conseq['Margem %'],
            'Estoque Consequente': info_conseq['Quantidade estoque'],
            
            'Suporte (%)': f"{regra['support']*100:.2f}%",
            'Confiança (%)': f"{regra['confidence']*100:.2f}%"
        })
    
    # Criar DataFrame final
    colunas = [
        'Antecedente (Código)', 'Descrição Antecedente', 'Valor Unitário Antecedente',
        'Margem % Antecedente', 'Estoque Antecedente',
        'Consequente (Código)', 'Descrição Consequente', 'Valor Unitário Consequente',
        'Margem % Consequente', 'Estoque Consequente',
        'Suporte (%)', 'Confiança (%)'
    ]
    
    return pd.DataFrame(dados_formatados, columns=colunas)


# Uso da função
regras_formatadas = formatar_regras_com_info(regras, caminho_arquivo)
display(regras_formatadas.head())


,Antecedente (Código),Descrição Antecedente,Valor Unitário Antecedente,Margem % Antecedente,Estoque Antecedente,Consequente (Código),Descrição Consequente,Valor Unitário Consequente,Margem % Consequente,Estoque Consequente,Suporte (%),Confiança (%)
0,36952,FILTRO LINHA 6 TOM BR TM104 OEX,26.30,0.31,0,1573,ESPIRAL 29MM PRETA 200F C/35 COPYART 75,24.21,0.30,122,11.11%,100.00%
1,36952,FILTRO LINHA 6 TOM BR TM104 OEX,26.30,0.31,0,1572,ESPIRAL 20MM PRETA 120F C/80 COPYART 72,25.49,0.26,207,11.11%,100.00%
2,1572,ESPIRAL 20MM PRETA 120F C/80 COPYART 72,25.49,0.26,207,36952,FILTRO LINHA 6 TOM BR TM104 OEX,26.30,0.31,0,11.11%,100.00%
3,1573,ESPIRAL 29MM PRETA 200F C/35 COPYART 75,24.21,0.30,122,36952,FILTRO LINHA 6 TOM BR TM104 OEX,26.30,0.31,0,11.11%,100.00%
4,36952,FILTRO LINHA 6 TOM BR TM104 OEX,26.30,0.31,0,4303,LIVRO PROTOCOLO 104F TILIBRA,13.32,0.32,1022,11.11%,100.00%


#### Diagnóstico geral sobre o produto pesquisado

In [4]:
print("\n=== Diagnóstico ===")
print(f"Total de produtos únicos: {len(caminho_arquivo['Código produto'].unique())}")
print(f"Total de transações únicas: {len(caminho_arquivo['Numero nota fiscal'].unique())}")

prod_transacoes = caminho_arquivo[caminho_arquivo['Código produto'] == codigo]
print(f"\nProduto {codigo} aparece em {len(prod_transacoes)} registros")
print(f"Em {len(prod_transacoes['Numero nota fiscal'].unique())} transações distintas")

prod_associados = caminho_arquivo[
    caminho_arquivo['Numero nota fiscal'].isin(prod_transacoes['Numero nota fiscal'])
]
print(f"\nProdutos associados em compras conjuntas:")
print(prod_associados['Código produto'].value_counts().head(10))


=== Diagnóstico ===
Total de produtos únicos: 5516
Total de transações únicas: 24008

Produto 36952 aparece em 9 registros
Em 9 transações distintas

Produtos associados em compras conjuntas:
Código produto
36952    9
36949    4
36947    4
36948    3
16560    3
36950    2
22053    2
33109    2
17770    2
27305    1
Name: count, dtype: int64
